# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../output_data/cities.csv")
weather_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Dwārka,23,IN,1586315915,81,22.24,68.97,78.26,7.52
1,Heimberg,0,CH,1586315916,93,46.79,7.60,46.00,1.12
2,Albany,90,US,1586315788,43,42.60,-73.97,55.40,5.82
3,Praia da Vitória,40,PT,1586315919,88,38.73,-27.07,62.60,17.16
4,Rodolfo Sánchez Taboada,80,MX,1586315920,80,31.72,-116.57,60.01,4.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = weather_df[["Lat", "Lng"]]
humidity = weather_df["Humidity"].astype(float)



In [4]:
gmaps.configure(api_key=g_key)

In [5]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=humidity.max(), point_radius=3)

fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
criteria_df = weather_df.loc[
    (weather_df["Max Temp"] > 70) & (weather_df["Max Temp"] < 80)
    & (weather_df["Wind Speed"] < 10) & (weather_df["Cloudiness"] == 0), :]
criteria_df = criteria_df.dropna(how='any')
criteria_df.reset_index(inplace=True)


del criteria_df["index"]
criteria_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Lázaro Cárdenas,0,MX,1586315946,80,17.96,-102.20,74.97,3.02
1,Salalah,0,OM,1586315781,50,17.02,54.09,75.20,4.70
2,Coahuayana Viejo,0,MX,1586315984,58,18.73,-103.68,74.70,2.95
3,Atar,0,MR,1586316201,19,20.52,-13.05,76.05,8.08
4,Oussouye,0,SN,1586316369,94,12.48,-16.55,73.40,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = criteria_df

hotels = []

for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    results = requests.get(base_url, params=params)
    jsn = results.json()
    try:
        hotels.append(jsn['results'][0]['name'])
    except:
        hotels.append("")
hotel_df["Hotel Name"] = hotels
hotel_df = criteria_df.dropna(how='any')
hotel_df.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,Lázaro Cárdenas,0,MX,1586315946,80,17.96,-102.20,74.97,3.02,Lázaro Cárdenas
1,Salalah,0,OM,1586315781,50,17.02,54.09,75.20,4.70,Salalah
2,Coahuayana Viejo,0,MX,1586315984,58,18.73,-103.68,74.70,2.95,Coahuayana de Hidalgo
3,Atar,0,MR,1586316201,19,20.52,-13.05,76.05,8.08,Atar
4,Oussouye,0,SN,1586316369,94,12.48,-16.55,73.40,4.70,Oussouye


In [8]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in criteria_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [9]:
hotel_name = hotel_df["Hotel Name"]
city = hotel_df["City"]
country = hotel_df["Country"]

info_box_template =(f"\
    Name : {hotel_name}\
    City : {city}\
    Country : {country}")

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in criteria_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))